Importando bibliotecas

In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA

Importando Base de Dados

In [3]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_cat.csv")
display(df.sample(5))

,TP_SEXO,TP_COR_RACA,TP_DEPENDENCIA_ADM_ESC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO,ESCOLARIDADE_PAI,ESCOLARIDADE_MAE,N_PESSOAS_CASA,RENDA_MENSAL_CASA,N_CARRO_CASA,N_MOTO_CASA,N_CELULAR_CASA,N_COMP_CASA,INTERNET_CASA,EST_IDADE
2046,Masculino,Parda,Privada,604.9,585.2,672.2,598.0,560.0,PG completo,EM incompleto,3,Até R$26400,Um,Um,Três,Três,True,17.0
7162,Feminino,Parda,Privada,564.0,510.3,554.7,543.5,680.0,EF-2 incompleto,EF-1 incompleto,3,Até R$1320,Nenhum,Nenhum,Um,Nenhum,True,18.0
6625,Feminino,Amarela,Estadual,401.6,415.3,353.4,439.6,540.0,ES incompleto,ES incompleto,3,Até R$1320,Um,Nenhum,Três,Nenhum,True,18.0
3825,Masculino,Branca,Privada,566.6,606.5,718.0,567.3,600.0,ES incompleto,ES incompleto,5,Até R$3300,Um,Um,Quatro ou mais,Um,True,18.0
2977,Feminino,Branca,Privada,626.9,601.0,715.3,578.6,900.0,ES incompleto,PG completo,4,Até R$3960,Um,Nenhum,Quatro ou mais,Quatro ou mais,True,18.0


Definindo funções que realizam testes de anova e manova

In [16]:
def anova_test(df:pd.DataFrame, y_col:str, x_cols:list[str]):
    ols_formula = f"{y_col} ~ {' * '.join(f'C({item})' for item in x_cols)}"
    print(ols_formula)
    ols_model = ols(formula=ols_formula, data=df).fit()
    display(sm.stats.anova_lm(ols_model, typ=2))

def anova_test_for_all_grades(df:pd.DataFrame, x_cols:list[str]):
    grade_cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
    for col in grade_cols:
        anova_test(df, col, x_cols)

def manova_test(df: pd.DataFrame, y_cols: list[str], x_col: str):
    manova_formula = f"{' + '.join(y_cols)} ~ {x_col}"
    print(manova_formula)
    manova_model = MANOVA.from_formula(manova_formula, data=df)
    mv_results = manova_model.mv_test()
    
    dfs = {}
    for factor, result in mv_results.results.items():
        stat_df = pd.DataFrame(result['stat'])
        stat_df.index.name = factor
        dfs[factor] = stat_df.reset_index()
    
    for factor, df_factor in dfs.items():
        print(factor)
        display(df_factor)

ANOVA - Sexo por Notas

In [15]:
anova_test_for_all_grades(df, ["TP_SEXO"])

NU_NOTA_CH ~ C(TP_SEXO)


,sum_sq,df,F,PR(>F)
C(TP_SEXO),2.583001e+04,1.0,3.937391,0.047251
Residual,6.558872e+07,9998.0,NaN,NaN


NU_NOTA_CN ~ C(TP_SEXO)


,sum_sq,df,F,PR(>F)
C(TP_SEXO),7.251339e+05,1.0,99.579094,2.423199e-23
Residual,7.280533e+07,9998.0,NaN,NaN


NU_NOTA_LC ~ C(TP_SEXO)


,sum_sq,df,F,PR(>F)
C(TP_SEXO),4.284760e+03,1.0,1.00313,0.316579
Residual,4.270537e+07,9998.0,NaN,NaN


NU_NOTA_MT ~ C(TP_SEXO)


,sum_sq,df,F,PR(>F)
C(TP_SEXO),3.474152e+06,1.0,219.500854,3.851687e-49
Residual,1.582434e+08,9998.0,NaN,NaN


NU_NOTA_REDACAO ~ C(TP_SEXO)


,sum_sq,df,F,PR(>F)
C(TP_SEXO),9.461076e+06,1.0,310.527496,1.806804e-68
Residual,3.046166e+08,9998.0,NaN,NaN


ANOVA - Cor/raça por nota

In [17]:
anova_test_for_all_grades(df, ["TP_COR_RACA"])

NU_NOTA_CH ~ C(TP_COR_RACA)


,sum_sq,df,F,PR(>F)
C(TP_COR_RACA),2.997721e+06,4.0,119.625272,7.341684e-100
Residual,6.261683e+07,9995.0,NaN,NaN


NU_NOTA_CN ~ C(TP_COR_RACA)


,sum_sq,df,F,PR(>F)
C(TP_COR_RACA),4.746496e+06,4.0,172.428354,4.808998e-143
Residual,6.878397e+07,9995.0,NaN,NaN


NU_NOTA_MT ~ C(TP_COR_RACA)


,sum_sq,df,F,PR(>F)
C(TP_COR_RACA),1.215672e+07,4.0,203.105331,9.212854e-168
Residual,1.495609e+08,9995.0,NaN,NaN


NU_NOTA_LC ~ C(TP_COR_RACA)


,sum_sq,df,F,PR(>F)
C(TP_COR_RACA),1.975357e+06,4.0,121.173687,3.872489e-101
Residual,4.073429e+07,9995.0,NaN,NaN


NU_NOTA_REDACAO ~ C(TP_COR_RACA)


,sum_sq,df,F,PR(>F)
C(TP_COR_RACA),1.231447e+07,4.0,101.969928,3.047033e-85
Residual,3.017633e+08,9995.0,NaN,NaN


ANOVA - Escolaridade mãe e escolaridade pai por Notas

In [20]:
anova_test_for_all_grades(df, ["ESCOLARIDADE_MAE", "ESCOLARIDADE_PAI"])

NU_NOTA_CH ~ C(ESCOLARIDADE_MAE) * C(ESCOLARIDADE_PAI)


c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 36, but rank is 35
  warnings.warn('covariance of constraints does not have full '


,sum_sq,df,F,PR(>F)
C(ESCOLARIDADE_MAE),1.623760e+06,6.0,46.351661,1.638747e-47
C(ESCOLARIDADE_PAI),2.265681e+06,6.0,64.675847,1.171065e-66
C(ESCOLARIDADE_MAE):C(ESCOLARIDADE_PAI),4.341841e+05,36.0,2.065694,2.202596e-04
Residual,5.810530e+07,9952.0,NaN,NaN


NU_NOTA_CN ~ C(ESCOLARIDADE_MAE) * C(ESCOLARIDADE_PAI)


c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 36, but rank is 35
  warnings.warn('covariance of constraints does not have full '


...

,sum_sq,df,F,PR(>F)
C(ESCOLARIDADE_MAE),1.005637e+06,6.0,43.578905,1.316880e-44
C(ESCOLARIDADE_PAI),1.306688e+06,6.0,56.624827,2.937091e-58
C(ESCOLARIDADE_MAE):C(ESCOLARIDADE_PAI),2.043773e+05,36.0,1.476102,3.479897e-02
Residual,3.827579e+07,9952.0,NaN,NaN


c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '


NU_NOTA_REDACAO ~ C(ESCOLARIDADE_MAE) * C(ESCOLARIDADE_PAI)


c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 5
  warnings.warn('covariance of constraints does not have full '
c:\Users\glaub\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 36, but rank is 35
  warnings.warn('covariance of constraints does not have full '


,sum_sq,df,F,PR(>F)
C(ESCOLARIDADE_MAE),5.751679e+06,6.0,33.221746,9.555806e-34
C(ESCOLARIDADE_PAI),7.991016e+06,6.0,46.156180,2.625819e-47
C(ESCOLARIDADE_MAE):C(ESCOLARIDADE_PAI),1.694333e+06,36.0,1.631081,1.077032e-02
Residual,2.871648e+08,9952.0,NaN,NaN


ANOVA - Dependencia da Escola por Nota

In [21]:
anova_test_for_all_grades(df, ["TP_DEPENDENCIA_ADM_ESC"])

NU_NOTA_CH ~ C(TP_DEPENDENCIA_ADM_ESC)


,sum_sq,df,F,PR(>F)
C(TP_DEPENDENCIA_ADM_ESC),6.667045e+06,3.0,376.853891,6.685670e-232
Residual,5.894750e+07,9996.0,NaN,NaN


NU_NOTA_CN ~ C(TP_DEPENDENCIA_ADM_ESC)


,sum_sq,df,F,PR(>F)
C(TP_DEPENDENCIA_ADM_ESC),1.066471e+07,3.0,565.24912,0.0
Residual,6.286576e+07,9996.0,NaN,NaN


NU_NOTA_MT ~ C(TP_DEPENDENCIA_ADM_ESC)


,sum_sq,df,F,PR(>F)
C(TP_DEPENDENCIA_ADM_ESC),2.543570e+07,3.0,621.885593,0.0
Residual,1.362819e+08,9996.0,NaN,NaN


NU_NOTA_LC ~ C(TP_DEPENDENCIA_ADM_ESC)


,sum_sq,df,F,PR(>F)
C(TP_DEPENDENCIA_ADM_ESC),3.638827e+06,3.0,310.322904,1.195791e-192
Residual,3.907082e+07,9996.0,NaN,NaN


NU_NOTA_REDACAO ~ C(TP_DEPENDENCIA_ADM_ESC)


,sum_sq,df,F,PR(>F)
C(TP_DEPENDENCIA_ADM_ESC),3.732579e+07,3.0,449.389899,6.565253e-274
Residual,2.767519e+08,9996.0,NaN,NaN


ANOVA - Renda mensal casa por Notas

In [22]:
anova_test_for_all_grades(df, ["RENDA_MENSAL_CASA"])

NU_NOTA_CH ~ C(RENDA_MENSAL_CASA)


,sum_sq,df,F,PR(>F)
C(RENDA_MENSAL_CASA),8.454672e+06,16.0,92.288286,8.459280e-284
Residual,5.715988e+07,9983.0,NaN,NaN


NU_NOTA_CN ~ C(RENDA_MENSAL_CASA)


,sum_sq,df,F,PR(>F)
C(RENDA_MENSAL_CASA),1.291369e+07,16.0,132.922489,0.0
Residual,6.061678e+07,9983.0,NaN,NaN


NU_NOTA_MT ~ C(RENDA_MENSAL_CASA)


,sum_sq,df,F,PR(>F)
C(RENDA_MENSAL_CASA),3.434157e+07,16.0,168.218404,0.0
Residual,1.273760e+08,9983.0,NaN,NaN


NU_NOTA_LC ~ C(RENDA_MENSAL_CASA)


,sum_sq,df,F,PR(>F)
C(RENDA_MENSAL_CASA),5.117374e+06,16.0,84.935569,1.191097e-261
Residual,3.759228e+07,9983.0,NaN,NaN


NU_NOTA_REDACAO ~ C(RENDA_MENSAL_CASA)


,sum_sq,df,F,PR(>F)
C(RENDA_MENSAL_CASA),3.147660e+07,16.0,69.495245,1.876379e-214
Residual,2.826011e+08,9983.0,NaN,NaN


ANOVA - Internet por Notas

In [23]:
anova_test_for_all_grades(df, ["INTERNET_CASA"])

NU_NOTA_CH ~ C(INTERNET_CASA)


,sum_sq,df,F,PR(>F)
C(INTERNET_CASA),4.366763e+05,1.0,66.984238,3.070456e-16
Residual,6.517787e+07,9998.0,NaN,NaN


NU_NOTA_CN ~ C(INTERNET_CASA)


,sum_sq,df,F,PR(>F)
C(INTERNET_CASA),5.207960e+05,1.0,71.318193,3.462651e-17
Residual,7.300967e+07,9998.0,NaN,NaN


NU_NOTA_MT ~ C(INTERNET_CASA)


,sum_sq,df,F,PR(>F)
C(INTERNET_CASA),1.571459e+06,1.0,98.10696,5.059057e-23
Residual,1.601461e+08,9998.0,NaN,NaN


NU_NOTA_LC ~ C(INTERNET_CASA)


,sum_sq,df,F,PR(>F)
C(INTERNET_CASA),3.611817e+05,1.0,85.270963,3.130277e-20
Residual,4.234847e+07,9998.0,NaN,NaN


NU_NOTA_REDACAO ~ C(INTERNET_CASA)


,sum_sq,df,F,PR(>F)
C(INTERNET_CASA),2.671263e+06,1.0,85.763428,2.445344e-20
Residual,3.114065e+08,9998.0,NaN,NaN
